# Arytmetyka komputerowa

## 1. Sumowanie liczb pojedynczej precyzji

In [1]:
#include <stdio.h>
#include <iostream>

#define MEASURE_TIME(FUNC) {\
    auto start = std::chrono::high_resolution_clock::now();\
    FUNC();\
    auto end = std::chrono::high_resolution_clock::now();\
    std::chrono::duration<double> time_span = std::chrono::duration_cast<std::chrono::duration<double>>(end - start);\
    std::cout << "time:" << time_span.count() << " seconds" << std::endl;\
}

In [2]:
std::vector<float> v;
const int N = 1e7;
const float x = 0.12345;
const float expectedResult = 1234500;

for (int i = 0; i < N; i++) {
    v.push_back(x);
}

### Wersja iteracyjna

In [3]:
void compute_array_sum() {
    float sum = 0;
    int i = 0;
    const int reportStep = 25000;

    for (auto num : v) {
        sum += num;
        i++;
    }

    float absoluteError = std::abs(expectedResult - sum);
    float relativeError = std::abs(expectedResult - sum) / expectedResult;

    std::cout << "iterative result: " << sum << std::endl;
    std::cout << "relative error: " << relativeError << std::endl;
    std::cout << "absolute error: " << absoluteError << std::endl;
}

In [4]:
MEASURE_TIME(compute_array_sum)

iterative result: 1.24951e+06
relative error: 0.0121596
absolute error: 15011
time:0.0849421 seconds


Wysoki błąd względny wynika z faktu, że podczas sumowania iteracyjego dodajemy do siebie liczby znacząco różniące się wielkością. Liczba *x* która wcześniej była reprezentowalna "znika" podczas dodawania, ponieważ po wyrównaniu z dużą liczbą obcinane się miejsca dziesiętne.

![graph](graph.png)

Błąd względny rośnie bardzo szybko na początku, potem wzrost ulega spowolnieniu.

### Wersja rekurencyjna

In [5]:
float recursive_sum(int l, int r) {
    if (l > r) {
        return 0;
    }

    if (l == r) {
        return x;
    }

    int mid = (l + r) / 2;
    return recursive_sum(l, mid) + recursive_sum(mid + 1, r);
}

In [6]:
void compute_recursive_sum() {
    float recursiveResult = recursive_sum(0, N);
    float absoluteError = std::abs(expectedResult - recursiveResult);
    float relativeError = std::abs(expectedResult - recursiveResult) / expectedResult;
    std::cout << "recursive result: " << recursiveResult << std::endl;
    std::cout << "absolute error: " << absoluteError << std::endl;
    std::cout << "relative error: " << relativeError << std::endl;
}

In [7]:
MEASURE_TIME(compute_recursive_sum)

recursive result: 1.2345e+06
absolute error: 0.125
relative error: 1.01256e-07
time:0.110238 seconds


Sumowanie metodą dziel i zwyciężaj ma mniejszy błąd względny, ponieważ dodawane sumy "połówkowe" mają podobną wielkość (sumę) - w przypadku tego zadania sumy "połówkowe" zawsze są równe.

## 2. Algorytm Kahana

In [8]:
float kahan_sum(std::vector<float> v) {
    float sum = 0;
    float err = 0;
    for (float x : v) {
        float y = x - err;
        float tmp = sum + y;
        err = (tmp - sum) - y;
        sum = tmp;
    }

    return sum;
}

In [9]:
void compute_kahan() {
    float res = kahan_sum(v);
    float absoluteError = std::abs(res - expectedResult);
    float relativeError = std::abs(res - expectedResult) / expectedResult;
    std::cout << "result: " << res << std::endl;
    std::cout << "absolute error: " << absoluteError << std::endl;
    std::cout << "relative error: " << relativeError << std::endl;
}

In [10]:
MEASURE_TIME(compute_kahan)

result: 1.2345e+06
absolute error: 0.125
relative error: 1.01256e-07
time:0.127863 seconds


Algorytm Kahana odzyskuje najmniej znaczące bity, które zostałyby utracone podczas sumowania liczb znacząco różniących się wielkością. Służy do tego zmienna *err*, która algebraicznie jest równa *0*, ale dlatego, że *tmp* jest zaokrąglona, zawiera utracone LSB które zostaną dodane w kolejnej iteracji.

Czasy wykonywania algorytmu Kahana i algorytmu sumowania rekurencyjnego są porównywalne dla tych danych wejściowych.

## 3. Suma szeregu

In [11]:
#include <cmath>

In [12]:
float float_sum(int n, bool reverse) {
    float exp =  1.0 / 4;
    std::vector<float> v;
    for (int i = 0; i < n; i++) {
        v.push_back(exp);
        exp /= 2;
    }

    float sum = 0;

    if (reverse) {
        for (int i = n - 1; i >= 0; i--) {
            sum += v[i];
        }
    } else {
        for (int i = 0; i < n; i++) {
            sum += v[i];
        }
    }

    return sum;
}

In [13]:
double double_sum(int n, bool reverse) {
    double exp =  1.0 / 4;
    std::vector<double> v;
    for (int i = 0; i < n; i++) {
        v.push_back(exp);
        exp /= 2;
    }

    double sum = 0;

    if (reverse) {
        for (int i = n - 1; i >= 0; i--) {
            sum += v[i];
        }
    } else {
        for (int i = 0; i < n; i++) {
            sum += v[i];
        }
    }

    return sum;
}

In [14]:
auto sizes = std::vector<int> { 50, 100, 200, 500, 800 };

In [21]:
std::cout.precision(10);
for (int size : sizes) {
    float resOrder = float_sum(size, false);
    float resReverse = float_sum(size, true);
    std::cout << "Size: " << size << std::endl;
    std::cout << "result (in order): " << resOrder << std::endl;
    std::cout << "absolute error: " << std::abs(0.5 - resOrder) << std::endl;
    std::cout << "relative error: " << std::abs(0.5 - resOrder) / 0.5 << std::endl;
    std::cout << "result (reversed): " << resReverse << std::endl;
    std::cout << "absolute error: " << std::abs(0.5 - resReverse) << std::endl;
    std::cout << "relative error: " << std::abs(0.5 - resReverse) / 0.5 << std::endl;
}

Size: 50
result (in order): 0.5000000000
absolute error: 0.0000000000
relative error: 0.0000000000
result (reversed): 0.5000000000
absolute error: 0.0000000000
relative error: 0.0000000000
Size: 100
result (in order): 0.5000000000
absolute error: 0.0000000000
relative error: 0.0000000000
result (reversed): 0.5000000000
absolute error: 0.0000000000
relative error: 0.0000000000
Size: 200
result (in order): 0.5000000000
absolute error: 0.0000000000
relative error: 0.0000000000
result (reversed): 0.5000000000
absolute error: 0.0000000000
relative error: 0.0000000000
Size: 500
result (in order): 0.5000000000
absolute error: 0.0000000000
relative error: 0.0000000000
result (reversed): 0.5000000000
absolute error: 0.0000000000
relative error: 0.0000000000
Size: 800
result (in order): 0.5000000000
absolute error: 0.0000000000
relative error: 0.0000000000
result (reversed): 0.5000000000
absolute error: 0.0000000000
relative error: 0.0000000000


In [23]:
std::cout.precision(10);
for (int size : sizes) {
    double resOrder = double_sum(size, false);
    double resReverse = double_sum(size, true);
    std::cout << "Size: " << size << std::endl;
    std::cout << "result (in order): " << resOrder << std::endl;
    std::cout << "absolute error: " << std::abs(0.5 - resOrder) << std::endl;
    std::cout << "relative error: " << std::abs(0.5 - resOrder) / 0.5 << std::endl;
    std::cout << "result (reversed): " << resReverse << std::endl;
    std::cout << "absolute error: " << std::abs(0.5 - resReverse) << std::endl;
    std::cout << "relative error: " << std::abs(0.5 - resReverse) / 0.5 << std::endl;
}

Size: 50
result (in order): 0.5000000000
absolute error: 0.0000000000
relative error: 0.0000000000
result (reversed): 0.5000000000
absolute error: 0.0000000000
relative error: 0.0000000000
Size: 100
result (in order): 0.5000000000
absolute error: 0.0000000000
relative error: 0.0000000000
result (reversed): 0.5000000000
absolute error: 0.0000000000
relative error: 0.0000000000
Size: 200
result (in order): 0.5000000000
absolute error: 0.0000000000
relative error: 0.0000000000
result (reversed): 0.5000000000
absolute error: 0.0000000000
relative error: 0.0000000000
Size: 500
result (in order): 0.5000000000
absolute error: 0.0000000000
relative error: 0.0000000000
result (reversed): 0.5000000000
absolute error: 0.0000000000
relative error: 0.0000000000
Size: 800
result (in order): 0.5000000000
absolute error: 0.0000000000
relative error: 0.0000000000
result (reversed): 0.5000000000
absolute error: 0.0000000000
relative error: 0.0000000000


In [30]:
for (int size : sizes) {
    float exp =  1.0 / 4;
    std::vector<float> vs;
    for (int i = 0; i < size; i++) {
        vs.push_back(exp);
        exp /= 2;
    }
    
    float sum = kahan_sum(vs);
    std::cout << "Size: " << size << std::endl;
    std::cout << "result: " << sum << std::endl;
}

Size: 50
result: 0.5000000000
Size: 100
result: 0.5000000000
Size: 200
result: 0.5000000000
Size: 500
result: 0.5000000000
Size: 800
result: 0.5000000000


W tym zadaniu otrzymałem dokładny wynik dla wszystkich przypadków, mimo, że według https://www.phys.uconn.edu/~rozman/Courses/P2200_11F/downloads/sum-howto.pdf (strona 2) sumowanie odwróconych liczb od tyłu powinno mieć o wiele mniejszy błąd.

## 4. Epsilon maszynowy

In [31]:
float compute_eps() {
    float eps = 0.5;
    float prev = 0;

    while (1 + eps != 1.0) {
        prev = eps;
        eps /= 2;
    }

    return prev;
}

In [33]:
compute_eps()

1.19209e-07f

Otrzymany wynik jest zgodny z matcheps podanym w standardzie IEEE 754.

## 5. Algorytm niestabilny numerycznie

Algorytm obliczania e^x

In [34]:
float compute_e(float exp) {
    float sum = 1;
    float lastsum = 0;
    float num = exp;
    long long fact = 1;
    long long i = 2;

    while (std::abs(std::abs(lastsum) - std::abs(sum)) > 1e-3) {
        lastsum = sum;
        sum += num / fact;
        num *= exp;
        fact = fact * i++;
    }

    return sum;
}

Przykład niestabilny:

In [35]:
compute_e(-5.5)

0.00387991f

Dla liczb ujemnych algorytm jest niestabilny, obliczenia są wykonywane dokładnie, jednak uzyskujemy wynik z wysokim błędem względnym. Wynika to z tego, że podczas obliczania występuje "catastrofic cancellation" - gdy odejmujemy liczby zbliżone do siebie, tracimy wiele znaczących bitów (w wyniku zaokrągleń), w rezultacie wynik odejmowania jest bardzo niedokładny.

Przykład stabilny (unikamy odejmowania):

In [36]:
1 / compute_e(5.5)

0.00408678f